# SQL Murder Mystery

Se ha cometido un delito y la policía necesita su ayuda. Un detective te dio el informe de la escena del crimen, pero de alguna manera lo perdiste. Recuerdas vagamente que el crimen fue un asesinato que ocurrió en algún momento del 15 de enero de 2018 y que tuvo lugar en SQL City. Empiece por recuperar el informe de la escena del crimen correspondiente de la base de datos del departamento de policía.

Requerido: Deberá de encontrar al asesino y también al culpable detrás de la planificación del crimen.

In [1]:
import sqlite3
from sqlite3 import Error

import pandas as pd

In [2]:
con = sqlite3.connect('sql-murder-mystery.db')

In [3]:
cursor = con.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('crime_scene_report',), ('drivers_license',), ('person',), ('facebook_event_checkin',), ('interview',), ('get_fit_now_member',), ('get_fit_now_check_in',), ('income',), ('solution',), ('sospechosos',)]


In [4]:
query1 = "SELECT * FROM crime_scene_report WHERE date = 20180115 AND type = 'murder' AND city = 'SQL City';"
murders = pd.read_sql(query1,con)
murders.iloc[0,2]

'Security footage shows that there were 2 witnesses. The first witness lives at the last house on "Northwestern Dr". The second witness, named Annabel, lives somewhere on "Franklin Ave".'

In [5]:
query2 = "SELECT * FROM person WHERE address_street_name = 'Northwestern Dr' ORDER BY address_number DESC LIMIT 1;"
personas1 = pd.read_sql(query2,con)
personas1.iloc[0,0]

14887

In [6]:
query3 = "SELECT * FROM 'person' WHERE address_street_name = 'Franklin Ave' AND Name LIKE 'Annabel%' ;"
personas2 = pd.read_sql(query3,con)
personas2.iloc[0,0]

16371

In [7]:
query4 = "SELECT * FROM 'interview' WHERE person_id =  14887 ;"
entrevista1 = pd.read_sql(query4,con)
entrevista1.iloc[0,1]

'I heard a gunshot and then saw a man run out. He had a "Get Fit Now Gym" bag. The membership number on the bag started with "48Z". Only gold members have those bags. The man got into a car with a plate that included "H42W".'

In [8]:
query5 = "SELECT * FROM 'interview' WHERE person_id =  16371 ;"
entrevista2 = pd.read_sql(query5,con)
entrevista2.iloc[0,1]

'I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th.'

In [9]:
query6 = "SELECT * FROM get_fit_now_member WHERE membership_status = 'gold' AND id LIKE '48Z%' limit 10;"
sospechosos = pd.read_sql(query6,con)
sospechosos

,id,person_id,name,membership_start_date,membership_status
0,48Z7A,28819,Joe Germuska,20160305,gold
1,48Z55,67318,Jeremy Bowers,20160101,gold


In [10]:
query7 = "SELECT * FROM 'get_fit_now_member' M INNER JOIN 'get_fit_now_check_in' C ON M.id = C.membership_id WHERE M.membership_status = 'gold' AND check_in_date = '20180109' AND id LIKE '48Z%' ;"
sospechosos2 = pd.read_sql(query7,con)
sospechosos2

,id,person_id,name,membership_start_date,membership_status,membership_id,check_in_date,check_in_time,check_out_time
0,48Z7A,28819,Joe Germuska,20160305,gold,48Z7A,20180109,1600,1730
1,48Z55,67318,Jeremy Bowers,20160101,gold,48Z55,20180109,1530,1700


In [11]:
query8 = "SELECT * FROM 'drivers_license' WHERE plate_number LIKE '%H42W%' AND gender = 'male' ;"
murders = pd.read_sql(query8,con)
murders

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS
1,664760,21,71,black,black,male,4H42WR,Nissan,Altima


In [12]:
query9 = "SELECT P.id,P.name,P.license_id,L.gender,L.plate_number FROM ('person' P INNER JOIN 'drivers_license' L ON P.license_id=L.id) WHERE plate_number LIKE '%H42W%' AND gender = 'male';"
entrevista2 = pd.read_sql(query9,con)
entrevista2

,id,name,license_id,gender,plate_number
0,51739,Tushar Chandra,664760,male,4H42WR
1,67318,Jeremy Bowers,423327,male,0H42W2


In [13]:
query10 = "SELECT P.id,P.name,P.license_id,L.gender,L.plate_number,GF.id,GF.membership_status FROM ('person' P INNER JOIN 'drivers_license' L ON P.license_id=L.id) INNER JOIN 'get_fit_now_member'GF ON P.id = GF.person_id WHERE plate_number LIKE '%H42W%' AND gender = 'male' AND membership_status='gold';"
entrevista2 = pd.read_sql(query10,con)
entrevista2

,id,name,license_id,gender,plate_number,id,membership_status
0,67318,Jeremy Bowers,423327,male,0H42W2,48Z55,gold


La unica persona que tiene un carro con una placa que incluye 'H42W', que sea hombre, y que sea parte del ginacio con una membrecia de oro, es Jeremy Bowers.
Eso quiere decir que el asesino es Jeremy Bowers.

Ahora hay que buscar lo que dijo en la entrevista:

In [14]:
cursor = con.cursor()
sql = "INSERT INTO solution VALUES (1, 'Jeremy Bowers')" 
cursor.execute(sql)
con.commit()
var = pd.read_sql("SELECT value FROM solution", con)
var.iloc[0,0]

"Congrats, you found the murderer! But wait, there's more... If you think you're up for a challenge, try querying the interview transcript of the murderer to find the real villian behind this crime. If you feel especially confident in your SQL skills, try to complete this final step with no more than 2 queries. Use this same INSERT statement with your new suspect to check your answer."

In [15]:
query11 = "SELECT * FROM 'interview' WHERE person_id =  67318 ;"
entrevista3 = pd.read_sql(query11,con)
entrevista3.iloc[0,1]

'I was hired by a woman with a lot of money. I don\'t know her name but I know she\'s around 5\'5" (65") or 5\'7" (67"). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.\n'

In [16]:
query5 = "SELECT P.name,P.id,F.person_id,Count(F.person_id) AS cantidad,F.event_name FROM ('person' P Inner Join 'drivers_license' L ON P.license_id=L.id INNER JOIN 'facebook_event_checkin' F ON P.id=F.person_id) WHERE L.gender = 'female' AND L.car_make = 'Tesla' AND L.car_model = 'Model S' AND L.hair_color = 'red' GROUP BY F.person_id HAVING cantidad = 3 ;"
entrevista2 = pd.read_sql(query5,con)
entrevista2.iloc[0,0]

'Miranda Priestly'

In [17]:
cursor = con.cursor()
sql = "INSERT INTO solution VALUES (1, 'Miranda Priestly')" 
cursor.execute(sql)
con.commit()
var = pd.read_sql("SELECT value FROM solution", con)
var.iloc[0,0]

'Congrats, you found the brains behind the murder! Everyone in SQL City hails you as the greatest SQL detective of all time. Time to break out the champagne!'